In [ ]:
import math
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from mpl_toolkits.axes_grid.inset_locator import inset_axes

from statsmodels.stats.proportion import proportion_confint

import pymc3
from scipy import stats
import sklearn as skl
from sklearn.metrics import roc_curve
from sklearn.calibration import calibration_curve

def iso_skill(x, y, br):
    unc=br*(1-br)
    return ((y-br)**2 - (x-y)**2)/unc

def get_ci_discrim(p,n):
    # Get CI for ratios (sens/spec/accuracy)
    z=1.96
    q = 1 - p
    ci_lower = (2 * n * p + z*z - 1 - z * math.sqrt(z*z - 2 - 1/n + 4 * p * (n * q + 1)))/(2 * (n + z*z))
    ci_upper = (2 * n * p + z*z + 1 + z * math.sqrt(z*z + 2 - 1/n + 4 * p * (n * q - 1)))/(2 * (n + z*z))
    se=(ci_upper-ci_lower)/3.92

    return (ci_lower,ci_upper),se

def get_ci_calib(stats):
    # get 95% CIs for calibration metrics
    alpha = 0.95
    p = ((1.0-alpha)/2.0) * 100
    ci_lower = max(0.0, np.percentile(stats, p))
    p = (alpha+((1.0-alpha)/2.0)) * 100
    ci_upper = min(1.0, np.percentile(stats, p))
    se=(ci_upper-ci_lower)/3.92

    return (ci_lower, ci_upper),se

def format_number(num):
    if abs(num)<0.001:
        x="0"
    else:
        x=str(float('%.2g' % num))
    return(x)

def calibration_loss(truth,forecast):
    # Calculates calibration loss and the Sanders score
    # Returns the array (needs to be summed later to get the number)
    
    # Assumes we don't need to re-bin)
    N=len(truth)
    levels=list(set(forecast))
    levels.sort()
    K=len(levels)
    calibration_loss_array=[]
    sanders_array=[]

    for pk in levels:
        nk=np.count_nonzero(forecast == pk)
        i=sum(np.where(forecast == pk))
        obark=sum(truth[i])/nk
        calibration_loss_array.append((nk/N)*((pk-obark)**2))
        sanders_array.append( (pk-obark)**2 + obark*(1-obark))

    return (calibration_loss_array,sanders_array)
    

def overconfidence_score(truth,forecast):
    
    # https://github.com/mlr-org/mlr/issues/842
    # Convert to numpy arrays
    forecast=np.array(forecast)
    truth=np.array(truth)
    
    # Generate the half-range forecasts 
    # i.e., Fold the array at 0.5 
    forecast_half_range=np.abs(forecast-0.5)+0.5
    
    # Get the forecast array ready for truth matching
    forecast[np.where(forecast<0.5)[0]]=0
    forecast[np.where(forecast>0.5)[0]]=1
    
    # Get the % correct
    perc_correct=len(np.where(abs(forecast-truth)==0)[0])/len(truth)
    
    return(np.mean(forecast_half_range)-perc_correct)

def plot_calib_results(truth, forecast, plot_ci=True, plot_isoskill=False, nplots=3, fig_title="",n_bins=20):


    # Get rid of too much precision in the forecasts 
    # We just need one decimal point here. This also helps
    # with binning later
    forecast=np.round(forecast,2)
    
    
    #####################################
    # Data for the plots
    # Calibration Curve
    calib_y, calib_x = calibration_curve(truth, forecast,n_bins=n_bins)
    # ROC curve
    fpr, tpr, thresholds = roc_curve(truth, forecast,pos_label=1)
    #####################################
    
    #####################################
    # Get 95% CI for the calibration plot
    # Get a list of the bins of calib_y
    #####################################
    # Get the counts for each item in calib_x
    bins = np.linspace(0., 1. + 1e-8, n_bins + 1)
    binids = np.digitize(forecast, bins) - 1
    bin_forecasts = np.bincount(binids, minlength=len(bins))
    bin_forecasts=bin_forecasts[bin_forecasts != 0]
    ci=[]
    i=0

    for prob in calib_y:
        p=prob
        n=bin_forecasts[i]
        AA=proportion_confint(count=n*p, nobs=n, alpha=0.05, method='binom_test')

        ci.append(AA)
        i=i+1
    # Transpose ci and subtract from midpoint so it'll work with pyplot.errorbar
    ci_adj=np.transpose(ci)
    ci_adj[0,]=calib_y-ci_adj[0,]
    ci_adj[1,]=ci_adj[1,]-calib_y
    #####################################
        
    #####################################
    # Get the metrics
    base_rate=sum(truth)/len(truth)
    uncertainty=base_rate*(1-base_rate)
    sharpness=np.var(forecast)

    # Calibration metrics
    brier=skl.metrics.brier_score_loss(truth,forecast)
    (temp_cl,sanders)=calibration_loss(truth=truth,forecast=forecast)
    calibration=sum(temp_cl)
    overconfidence=overconfidence_score(truth,forecast)
    cil=np.mean(forecast)-np.mean(truth)
    resolution=calibration+uncertainty-brier
    refinement=brier-calibration
    brier_base=uncertainty
    brier_skill_score=-(brier-brier_base)/brier_base
    min_bss=(uncertainty-1)/uncertainty
    kappa=skl.metrics.cohen_kappa_score(truth, forecast.round())


    # Discrimination metrics
    AUC=skl.metrics.auc(fpr,tpr)
    cm=skl.metrics.confusion_matrix(truth,forecast.round())
    tp=cm[1,1]
    tn=cm[0,0]
    fp=cm[0,1]
    fn=cm[1,0]
    sensitivity=tp/(tp+fn)
    specificity=tn/(tn+fp)
    accuracy=(tp+tn)/(len(truth))
    
    # Get CI and SE using frequentist method
    sensitivity_ci_f, sensitivity_se=get_ci_discrim(sensitivity,len(truth))
    specificity_ci_f, specificity_se=get_ci_discrim(specificity,len(truth))
    accuracy_ci_f, accuracy_se=get_ci_discrim(accuracy,len(truth))

    ###############################
    # Bootstrap CIs
    # configure bootstrap
    n_iterations = 1000

    # Define arrays to collect bootstraps
    uncertainty_a =list()
    brier_a =list()
    calibration_a =list()
    overconfidence_a =list()
    cil_a=list()
    resolution_a=list()
    refinement_a=list()
    brier_skill_score_a=list()
    auc_a=list()
    kappa_a=list()

    sensitivity_a=list()
    specificity_a=list()
    accuracy_a=list()


    values=np.column_stack((forecast,truth))
    data_size = len(truth) # find length of the data set
    
    for x in range(0, n_iterations-1):

        # create an array of random indices equal to data_size
        # this array will be used to sample with replacement from the arrays
        idx = np.random.randint(data_size, size=data_size)
        truth_s = truth[idx]

        # sample from forecasts
        forecast_s = forecast[idx]

        base_rate_s=sum(truth_s)/len(truth_s)
        uncertainty_s=base_rate_s*(1-base_rate_s)

        if uncertainty_s>0:

            brier_s=skl.metrics.brier_score_loss(truth_s,forecast_s)
            (temp_cl_s,sanders_s)=calibration_loss(truth_s,forecast_s)
            calibration_s=sum(temp_cl_s)
            overconfidence_s=overconfidence_score(truth_s,forecast_s)
            cil_s=np.mean(forecast_s)-np.mean(truth_s)
            resolution_s=calibration_s+uncertainty_s-brier_s
            refinement_s=brier_s-calibration_s
            brier_base_s=uncertainty_s
            brier_skill_score_s=-(brier_s-brier_base_s)/brier_base_s
            kappa_s=skl.metrics.cohen_kappa_score(truth_s,forecast_s.round())

            fpr_s, tpr_s, thresholds_s = roc_curve(truth_s, forecast_s,pos_label=1)
            auc_s=skl.metrics.auc(fpr_s,tpr_s)

            cm_s=skl.metrics.confusion_matrix(truth_s,forecast_s.round())
            tp_s=cm[1,1]
            tn_s=cm[0,0]
            fp_s=cm[0,1]
            fn_s=cm[1,0]
            sensitivity_s=tp_s/(tp_s+fn_s)
            specificity_s=tn_s/(tn_s+fp_s)
            accuracy_s=(tp_s+tn_s)/(len(truth_s))
            uncertainty_a.append(uncertainty_s)
            brier_a.append(brier_s)
            calibration_a.append(calibration_s)
            overconfidence_a.append(overconfidence_s)
            cil_a.append(cil_s)
            resolution_a.append(resolution_s)
            refinement_a.append(refinement_s)
            brier_skill_score_a.append(brier_skill_score_s)
            auc_a.append(auc_s)
            sensitivity_a.append(sensitivity_s)
            specificity_a.append(specificity_s)
            accuracy_a.append(accuracy_s)
            kappa_a.append(kappa_s)

        else:
            x=x-1

    # Get CI and SE using frequentist method
    brier_ci,brier_se=get_ci_calib(brier_a)
    calibration_ci,calibration_se=get_ci_calib(calibration_a)
    overconfidence_ci,overconfidence_se=get_ci_calib(overconfidence_a)
    cil_ci,cil_se=get_ci_calib(cil_a)
    resolution_ci,resolution_se=get_ci_calib(resolution_a)
    refinement_ci,refinement_se=get_ci_calib(refinement_a)
    brier_skill_score_ci,brier_skill_score_se=get_ci_calib(brier_skill_score_a)
    auc_ci,auc_se=get_ci_calib(auc_a)
    kappa_ci,kappa_se=get_ci_calib(kappa_a)

    
    # Get CI using 95%HPD
    uncertainty_ci=pymc3.stats.hpd(np.array(uncertainty_a))
    brier_ci=pymc3.stats.hpd(np.array(brier_a))
    calibration_ci=pymc3.stats.hpd(np.array(calibration_a))
    overconfidence_ci=pymc3.stats.hpd(np.array(overconfidence_a))
    resolution_ci=pymc3.stats.hpd(np.array(resolution_a))
    refinement_ci=pymc3.stats.hpd(np.array(refinement_a))
    brier_skill_score_ci=pymc3.stats.hpd(np.array(brier_skill_score_a))
    auc_ci=pymc3.stats.hpd(np.array(auc_a))
    sensitivity_ci=pymc3.stats.hpd(np.array(sensitivity_a))
    specificity_ci=pymc3.stats.hpd(np.array(specificity_a))
    accuracy_ci=pymc3.stats.hpd(np.array(accuracy_a))
    kappa_ci=pymc3.stats.hpd(np.array(kappa_a))


    # Get Zombie plot ellipse coordinates
    ellipse_width=specificity_ci_f[1]-specificity_ci_f[0]
    ellipse_height=sensitivity_ci_f[1]-sensitivity_ci_f[0]
    ellipse_center=[]
    ellipse_center=[1-(specificity_ci_f[0]+ellipse_width/2),sensitivity_ci_f[0]+ellipse_height/2]

    #####################################
    
    if plot_isoskill:
        # Get equal skill lines
        ls = np.linspace(0.0,1.0,100)
        X, Y = np.meshgrid(ls,ls)
        Z = iso_skill(X,Y,base_rate)

    #####################################
    # Plots
    # Set figure defaults
    fig_height=6
    fig_width=13

    if (nplots==3):
        f, axs = plt.subplots(1,3,figsize=(fig_width,fig_height))
        title_size=14
        axis_size=13
        tick_size=12
        marker_size=7
        line_width=2
        auc_font_size=12
        sup_y=0.9
    else:
        f, axs = plt.subplots(1,2,figsize=(fig_width,fig_height))
        title_size=18
        axis_size=16
        tick_size=14
        marker_size=10
        line_width=2.5
        auc_font_size=16
        sup_y=1.03

    f.suptitle(fig_title, fontsize=24, ha="center",y=sup_y)

    ###########################################################################
    # Plot the confidence  calibration curve
    axs[0].errorbar(calib_x,calib_y,yerr=ci_adj,fmt='o-',capsize=4,markersize=marker_size,linewidth=line_width)

    # Plot the diagnonal
    axs[0].plot([0,1],[0,1],'k-',linewidth=line_width/2)
    # Plot the base rate
    axs[0].plot([0,1],[base_rate,base_rate],'k--',linewidth=line_width/2)
    # Plot the skill area
    x1=[0,base_rate,1]
    y1=[base_rate/2., base_rate,(1+base_rate)/2.]
    axs[0].fill_between(x1[0:2],0,y1[0:2], color="green",alpha=0.1)
    axs[0].fill_between(x1[1:3],y1[1:3],1, color="green",alpha=0.1)

    # Plot lines of optimal resolution
    axs[0].plot([0,base_rate],[0,0],'g:')
    axs[0].plot([base_rate,1],[1,1],'g:')

    # Plot lines of optimal accuracy
    epsilon=0.0
    axs[0].plot([0,0.5],[0+epsilon,0+epsilon],'g-.')
    axs[0].plot([0.5,1],[1-epsilon,1-epsilon],'g-.')

    # Plot isoskill lines if needed
    if plot_isoskill:
        # Find minimum skill given the prevalence. 
        # Used to get a reasonable range of contours
        min_skill=round((uncertainty-1)/uncertainty,1)

        # Set floor at -10, since it can get to -inf at low uncertainty
        if (min_skill<-10):
            min_skill=-10

        # Get a rounded step size from min skill to 0
        interval=(0-min_skill)/5
        # Create a range from min_skill to zero
        neg_interval=np.round(np.arange(min_skill,0,interval),0)
        # Add positive range of skill from 0 to 1, in increments of 0.2
        combined_interval=np.unique(np.concatenate((neg_interval,np.arange(0,1.1,0.2))))

        # Create the iso-contour lines and add to axs[0]
        contours = axs[0].contour(X,Y,Z, combined_interval, colors="green", alpha=0.7)
        axs[0].clabel(contours, inline=True, fontsize=8)


    #########################################################################################
    # Plot the sharpness histogram
    # Set location of sharpness histogram under the confidence calibration curve
    if nplots==3:
        ax_inset=inset_axes(axs[0], width="100%",  height="40%", bbox_to_anchor=(-0.025,-0.7,1,.65), loc=2, bbox_transform=axs[0].transAxes)
    else:
        ax_inset=inset_axes(axs[0], width="100%",  height="40%", bbox_to_anchor=(-0.015,-0.7,1,.65), loc=2, bbox_transform=axs[0].transAxes)
    
    # Set the width of the bins. If there's only one bin and set the width at 0.1
    if (len(calib_x)>1):
        bar_width=min([0.1,min(np.ediff1d(calib_x))])
    else:
        bar_width=0.1

    # Plot the sharpness histogram
    ax_inset.bar(calib_x,bin_forecasts,width=bar_width)

    # Print the text labels for the sharpness histogram
    for i in range(len(calib_x)):
        ax_inset.text(calib_x[i], bin_forecasts[i] + max(bin_forecasts)*0.07, str(bin_forecasts[i]), color='black',ha='center')
    
    # Set labels and limits for the confidence calibration curve and sharpness histogram
    axs[0].set_title('Confidence Calibration Curve',fontsize=title_size)
    axs[0].set_ylabel('Observed probability of event',fontsize=axis_size)
    axs[0].yaxis.set_label_coords(-0.15,0.5)
    axs[0].set_aspect('equal', adjustable='box')
    axs[0].set_xlim(-0.05,1.05)
    axs[0].set_ylim(-0.05,1.05)
    axs[0].set_xticklabels([])
    axs[0].tick_params(axis="x", labelsize=tick_size)
    axs[0].tick_params(axis="y", labelsize=tick_size)
    ax_inset.set_xlabel('Predicted chance of event',fontsize=axis_size)
    ax_inset.set_ylabel('# predictions',fontsize=axis_size,labelpad=133)
    ax_inset.set_xlim(-0.05,1.05)
    ax_inset.set_ylim(0,max(bin_forecasts)*1.35)
    ax_inset.yaxis.set_label_coords(-0.15,0.5)
    ax_inset.tick_params(axis="x", labelsize=tick_size)
    ax_inset.tick_params(axis="y", labelsize=tick_size)

    ###########################################################################
    # Plot the ROC
    axs[1].plot(fpr, tpr,'o-',markersize=marker_size,linewidth=line_width)
    # Plot the diagnonal
    d_x=[0, 0.33333, 1]
    d_y=[0, 0.33333, 1]
    axs[1].plot(d_x, d_y,'k--',linewidth=line_width/2)
    # Shift ROC subplot to make it look nicer
    box = axs[1].get_position()
    box.x0 = box.x0 + 0.01
    box.x1 = box.x1 + 0.01
    axs[1].set_position(box)

    # Plot the zombie plot areas
    z_x1=[0, 0.33333,1]
    z_y1=[0, 1,1]
    z_x2=[0,1,1]
    z_y2=[0.6666,1,1]
    axs[1].fill_between(z_x1,z_y1, d_y ,color="red",alpha=0.1)
    axs[1].fill_between(z_x2,z_y2, 0,color="red",alpha=0.1)
    axs[1].fill_between(d_x,d_y, 0,color="red",alpha=0.3)
    auc_text='AUC='+str(AUC.round(3))
    axs[1].text(0.48,0.25,auc_text,fontsize=auc_font_size)
    axs[1].add_artist(Ellipse(ellipse_center, ellipse_width, ellipse_height,color="black",alpha=0.1))
    axs[1].scatter(1-specificity,sensitivity,marker="s",s=marker_size*15,color="black")

    # Set labels and limits for ROC curve
    axs[1].set_title('ROC Curve',fontsize=title_size)
    axs[1].set_xlabel('1-Specificity',fontsize=axis_size)
    axs[1].set_ylabel('Sensitivity',fontsize=axis_size)
    axs[1].set_aspect('equal', adjustable='box')
    axs[1].tick_params(axis="x", labelsize=tick_size)
    axs[1].tick_params(axis="y", labelsize=tick_size)


    #####################################################################################################################
    # Put the output variables in a dict
    #  Format: Name, value, 95%HDI low, 95%HDI high, SE, range low, range high, explanation, direction of improved performance
    metrics=dict([('Prevalence', (base_rate,False,False,False,0,1,"Pre-test probability based on truth.",False)), 
                  ('Uncertainty', (uncertainty,uncertainty_ci[0],uncertainty_ci[1],False,0,0.25,"Lower indicates an easier diagnostic problem.",False)), 
                  ('Sharpness', (sharpness,False,False,False,0,0.25,"Measure of variance of the forecasts. No relationship to quality of forecasts.",False)),
                  ('Calibration loss', (calibration,calibration_ci[0],calibration_ci[1],calibration_se,0,1,"How well forecasts line up with the diagonal. Lower is better.","low")), 
                  ('Confidence score', (overconfidence,overconfidence_ci[0],overconfidence_ci[1],overconfidence_se,0,1,"Degree of confidence. Negative and positive values indicate under- and over-confidence, respectively. A score of 0 is best ","low")), 
                  ('Calibration-in-large', (cil,cil_ci[0],cil_ci[1],cil_se,0,1,"Mean of forecasts-mean of outcomes. A score of 0 is best ","low")), 
                  ('Resolution', (resolution,resolution_ci[0],resolution_ci[1],resolution_se,0,round(uncertainty,2),"How well the forecasts are differentiated from the prevalence. Higher is better. Upper value =  uncertainty","high")), 
                  ('Refinement loss', (refinement,refinement_ci[0],refinement_ci[1],refinement_se,0,round(uncertainty,2),"How poorly predictions discriminate between the presence and absence of disease. Lower is better. Upper limit = uncertainty.","low")),
                  ('Cohen Kappa', (kappa,kappa_ci[0],kappa_ci[1],kappa_se,0,1,"Agreement between forecast and truth after accounting for randomness. Upper is better.","high")),
                  ('Brier score loss', (brier,brier_ci[0],brier_ci[1],brier_se, 0,1,"Measure of the error between predictions and truth. Lower is better.","low")), 
                  ('Brier skill score', (brier_skill_score,brier_skill_score_ci[0],brier_skill_score_ci[1],brier_skill_score_se,round(min_bss,2),1,"Measures the skill of the predictor compared to guessing based on pre-test probability. Lowest (worst) value is a function of uncertainty and ranges from -infinity at uncertainty of 0 to -3 at uncertainty of 0.25. Higher is better.","high")), 
                  ('Sensitivity', (sensitivity,sensitivity_ci_f[0],sensitivity_ci_f[1],sensitivity_se,0,1,"True positive rate. Measures the proportion of actual positives that are correctly identified as such. Higher is better.","high")), 
                  ('Specificity', (specificity,specificity_ci_f[0],specificity_ci_f[1],specificity_se,0,1,"True negative rate. Measures the proportion of actual negatives that are correctly identified as such. Higher is better.","high")), 
                  ('Accuracy', (accuracy,accuracy_ci_f[0],accuracy_ci_f[1],accuracy_se, 0,1,"True classification rate. Measures the proportion of actual positive and negatives that are correctly identified as such. Higher is better.","high")), 
                  ('AUC',(AUC,auc_ci[0],auc_ci[1],auc_se,0,1,"Area under the ROC curve. Probability of ranking a randomly chosen positive case higher than a randomly chosen negative one. Higher is better.","high"))
])
    
    if (nplots==3):
        # Generate text block for plot3 
        h='Metric\t\t\tValue\t95%HDI/CI\n'
        h=h+'-----------------------------------------\n'
        t=''
        for m in metrics:  
            temp_array=metrics[m]
            tab_1="\t"
            if len(m)<16:
                tab_1=tab_1+"\t"
                if len(m)<4:
                    tab_1=tab_1+"\t"
            if temp_array[2]==False:
                t=t+m+tab_1+format_number(temp_array[0])+"\n"
            else:
                t=t+m+tab_1+format_number(temp_array[0])+"\t"+format_number(temp_array[1])+" - "+ format_number(temp_array[2])+"\n"
        
        t=t+'-----------------------------------------'

        # Print text on plot 3 and turn off its axes
        axs[2].axis('off')
        axs[2].set_aspect('equal', adjustable='box')
        axs[2].text(-0.10,0.85,h.expandtabs(),fontproperties='monospace',fontsize=9,fontweight='bold',verticalalignment='bottom')
        axs[2].text(-0.10,0.9,t.expandtabs(),fontproperties='monospace',fontsize=9,verticalalignment='top')
        axs[2].tick_params(axis="x", labelsize=tick_size)
        axs[2].tick_params(axis="y", labelsize=tick_size)


    return (plt,metrics,cm)


#####################################